##　セグメンテーション
9から12章では分類機のモデル訓練に使用するデータは人手でアノテーションしている。
モデルへの入力の結節を自動で生成したい。

現実世界のアプローチでは複数の問題を個々のステップで解決するが、ディープラーニングの研究では複数の問題から構成されている問題を単一のモデルによって解決させる傾向がある。

１３章ではCTスキャンの元データから結節である可能性がある領域を全て見つける。
結節の一部であるボクセルに対してラベルをつけるセグメンテーション処理を行う。

セマンティックセグメンテーション

画像内の個々のピクセルに対してラベルをつける。今回は結節にTrue、正常組織にFalseのラベルをつける。


物体検出

画像内の対象にバウンディングボックスを設定する
こちらの方が計算リソースが必要である

## U-NET

分類タスクでは画像を畳み込みとダウンサンプリングを繰り返し、各クラスの確率のベクトルにする。

セグメンテーションでは入力と出力のサイズは同じにしたい。畳み込みによりテクスチャや色を検出し、ダウンサプリングによって畳み込みの受容野を広げ局所だけでなく広い特徴を掴む。このようにしていくと画像サイズが小さくなっていくので、1つのピクセルをn＊nのブロックに置き換えるアップサンプリングを行う。

さらに今回はパディングをおこなことで、画像周辺のピクセルが失われないようにして、かつUのダウンサンプリング時とアップサンプリング時のサイズが同じになる。

スキップ接続がないと、ダウンサンプリング時に画像が小さくなり物体境界の正確な位置情報が失われやすくなる。

U-Netのスキップ接続はResNetのスキップ接続とは異なり、ダウンサンプリング側の入力を対応する出力側のアップサンプリング側へつなぐ。
このことにより、Uの底で広い受容野の情報とネットワーク初期の入力に近い高精細な情報の両方を出力へ繋げていく。

既存コードの再利用は良いアイデアだが、どのようなモデルで、どのような実装、訓練か取り組んでいるプロジェクトに適用できる部分があるか把握しておくことが必要。

既存のモデルを変更していく場合は、1つずつ変更し比較していくと良い。

今回は既存のU-NETから
1：入力をバッチ正規化する
2：出力の前にnn.sigmoidを使い[0,1]の範囲にする
3：モデルの深さとフィルタを減らす

In [6]:
class UNetWrapper(nn.Module):
    def __init__(self, **kwargs):
        # kwargs はコンストラクタに渡される全てのキーワード引数を含む辞書
        super().__init__()

        # BatchNorm2d は入力のチャンネル数を必要とする
        # その情報をキーワード引数から取り出す
        self.input_batchnorm = nn.BatchNorm2d(kwargs["in_channels"])
        # U-Netの取り込み部分はこれだけだが、ほとんどの処理はここで行われる
        self.unet = UNet(**kwargs)
        self.final = nn.Sigmoid()

        # 第11章と同じように独自の重み初期化を行う
        self._init_weights()
        
    def _init_weights(self):
        init_set = {
            nn.Conv2d,
            nn.Conv3d,
            nn.ConvTranspose2d,
            nn.ConvTranspose3d,
            nn.Linear,
        }
        for m in self.modules():
            if type(m) in init_set:
                nn.init.kaiming_normal_(
                    m.weight.data, mode="fan_out", nonlinearity="relu", a=0
                )
                if m.bias is not None:
                    fan_in, fan_out = nn.init._calculate_fan_in_and_fan_out(
                        m.weight.data
                    )
                    bound = 1 / math.sqrt(fan_out)
                    nn.init.normal_(m.bias, -bound, bound)

        # nn.init.constant_(self.unet.last.bias, -4)
        # nn.init.constant_(self.unet.last.bias, 4)

    def forward(self, input_batch):
        bn_output = self.input_batchnorm(input_batch)
        un_output = self.unet(bn_output)
        fn_output = self.final(un_output)
        return fn_output



NameError: name 'nn' is not defined

今回の画像は三次元だが、nn.Batchnormは2d
メモリ使用量を減らすためだが、前後の画像の情報は検出には必要。
二次元画像として処理してセグメンテーション処理時には三次元画像として渡す。

モデル学習時に自ら隣接していることを学ぶ必要があるが、z軸の画像量が少なく容易と考える。

画像のチャネルをスライス（＋2、＋1、0、-1、-2）に絞り、スライス、x軸、y軸の入力とする。
スライス方向の情報を限定的にするが、結節の大きさは小さく今回の問題では十分だと判断。

## モデルの設計

どのようなトレードオフを考えなけらばいけないかはフローチャートや経験則はないが、体系的に仮説を検証することが大切であり、
思いつきの変更等は堪え、複数の変更を同時にテストすることはだめ。

## 正解データの作成

1：バウンディングボックス

結節の中心の位置はわかっているから、閾値以下になるまで左右、上下に探索し閾値以下になったらそこまでの範囲とする。
他の組織と隣接している可能性もあるので、片方が低密度に触れたら探索終了となる。片方ずつの独立の探索はできず、結節は中心の情報が必要。

下記のループ処理後にバウンディングボックス内の閾値より高い領域を調理和として取り出し、マスクとする。

In [ ]:
def buildAnnotationMask(self, positiveInfo_list, threshold_hu=-700):
        # hu_aと同じ次元のゼロarrayを作成
        boundingBox_a = np.zeros_like(self.hu_a, dtype=np.bool)

        for candidateInfo_tup in positiveInfo_list:
            center_irc = xyz2irc(
                candidateInfo_tup.center_xyz,
                self.origin_xyz,
                self.vxSize_xyz,
                self.direction_a,
            )
            
            # 結節中心の位置情報
            ci = int(center_irc.index)
            cr = int(center_irc.row)
            cc = int(center_irc.col)

            index_radius = 2
            try:
                while (
                    self.hu_a[ci + index_radius, cr, cc] > threshold_hu
                    and self.hu_a[ci - index_radius, cr, cc] > threshold_hu
                ):
                    index_radius += 1
            except IndexError:
                index_radius -= 1
                
            row_radius = 2
            try:
                while (
                    self.hu_a[ci, cr + row_radius, cc] > threshold_hu
                    and self.hu_a[ci, cr - row_radius, cc] > threshold_hu
                ):
                    row_radius += 1
            except IndexError:
                row_radius -= 1

            col_radius = 2
            try:
                while (
                    self.hu_a[ci, cr, cc + col_radius] > threshold_hu
                    and self.hu_a[ci, cr, cc - col_radius] > threshold_hu
                ):
                    col_radius += 1
            except IndexError:
                col_radius -= 1
                
                
            
            boundingBox_a[
                ci - index_radius : ci + index_radius + 1,
                cr - row_radius : cr + row_radius + 1,
                cc - col_radius : cc + col_radius + 1,
            ] = True
        
        # バウンディングボックスからマスクをくり抜く
        mask_a = boundingBox_a & (self.hu_a > threshold_hu)

        return mask_a

In [51]:
import numpy as np
x = np.ones(27)
x = x.reshape((3, 3, 3))
x

a=np.zeros_like(x,dtype=np.bool)
a

/tmp/ipykernel_221/1484239419.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  a=np.zeros_like(x,dtype=np.bool)


array([[[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]]])

In [52]:
a[1:2, 0:2, 1:2]=True

In [53]:
a

array([[[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False,  True, False],
        [False,  True, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]]])